크롤링 데이터 기간: 2008년 4월 ~ 2020년 7월(9월 7일 기준 최신데이터)  
dates/titles/contents 세개의 데이터를 데이터프레임으로 생성해 저장한다.

In [6]:
# !pip install selenium
# !pip install tika
# !pip install olefile
import os
import re
import datetime
import pandas as pd
import numpy as np
from selenium import webdriver
from tika import parser
from bs4 import BeautifulSoup
import requests
# import olefile # hwp파일 전체 안읽어지는 문제 발생

In [ ]:
# 방법2-2,자료(pdf)를 로컬에 다운받고 pdf를 변환한다.
# url 가져올 필요 없이 바로 다운로드 받는다.
# driver = webdriver.Chrome('./chromedriver.exe')

last_page = 26
pages = np.arange(1,last_page +1)

path = "C:\\Users\\user\\MPB_minutes"

if not os.path.exists(path):
    os.makedirs(path)

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
prefs = {"profile.default_content_settings.popups": 0,
             "download.default_directory": 
                        path,
             "directory_upgrade": True}
options.add_experimental_option("prefs", prefs)
driver=webdriver.Chrome('./chromedriver.exe', options=options)

for page in pages:
    url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(page)
    driver.get(url)
    
    li_tags = driver.find_elements_by_css_selector('div.type2 li a.secretSet')
    n_li_tags = len(li_tags)
    for idx in range(n_li_tags):
        li_tag = li_tags[idx]
        li_tag.click()
        name = driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li:nth-child(1) > a:nth-child(1)').text
        name = driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li:nth-child(1) > a:nth-child(1)').text
        if name.split('.')[1] == 'hwp':
            try:
                driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li:nth-child(2) > a:nth-child(1)').click()
            except:
                driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li > a:nth-child(1)').click()
        else:
            driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li:nth-child(1) > a:nth-child(1)').click()
        driver.back()
        li_tags = driver.find_elements_by_css_selector('div.type2 li a.secretSet') # 페이지가 새로 로드되면 li_tags를 잃어버리므로 다시 할당
#         li_tags.clear()
#         driver.implicitly_wait(10)
# driver.close()

In [7]:
# hwp -> text로 변환시키기
# 1. olefile - 1장만 읽어오는 문제 발생
# 2. tika 사용

#로컬 파일의 파일명 리스트를 가져온다.

path = "C:/Users/user/MPB_minutes/"
file_list = os.listdir(path)

file_list_pdf = [file for file in file_list if file.endswith(".pdf")]
file_list_hwp = [file for file in file_list if file.endswith(".hwp")]

file_list = file_list_pdf + file_list_hwp
print(file_list[:10], len(file_list), sep='\n')

['munete20.pdf', '금융통화위원회 의사록(2008년도 제10차).pdf', '금융통화위원회 의사록(2008년도 제11차).pdf', '금융통화위원회 의사록(2008년도 제12차).pdf', '금융통화위원회 의사록(2008년도 제13차).pdf', '금융통화위원회 의사록(2008년도 제14차).pdf', '금융통화위원회 의사록(2008년도 제15차).pdf', '금융통화위원회 의사록(2008년도 제16차).pdf', '금융통화위원회 의사록(2008년도 제18차).pdf', '금융통화위원회 의사록(2008년도 제19차).pdf']
255


In [34]:
dates = []
contents = []
titles = []

i = 1
for idx, file in enumerate(file_list):
    
    raw = parser.from_file('C:/Users/user/MPB_minutes/{}'.format(file))
    date2 = re.search('\d{4}[가-힣\s]+\d+월? \d+일?', raw['content']).group()
    date2 = '.'.join(re.findall('[0-9]+',date2))
        
    title = file.split('.')[0]
    
#     # 파일명이 중복되는 경우, 중복 숫자 제거
#     if list(title.split(' ')[-1])[-1] == ')':
#         title = ' '.join(title.split(' ')[:-1])
    dates.append(date2)
    titles.append(title)
    contents.append(raw['content'])

    i += 1
    if i % 10 == 0:
        print(i,'/',len(file_list), '완료')
print('모든 작업 완료')

10 / 255 완료
20 / 255 완료
30 / 255 완료
40 / 255 완료
50 / 255 완료
60 / 255 완료
70 / 255 완료
80 / 255 완료
90 / 255 완료
100 / 255 완료
110 / 255 완료
120 / 255 완료
130 / 255 완료
140 / 255 완료
150 / 255 완료
160 / 255 완료
170 / 255 완료
180 / 255 완료
190 / 255 완료
200 / 255 완료
210 / 255 완료
220 / 255 완료
230 / 255 완료
240 / 255 완료
250 / 255 완료
모든 작업 완료


In [35]:
df = pd.DataFrame({'date':dates,'title':titles,'content':contents})

df.date = pd.to_datetime(df.date)
df = df.sort_values(by='date',ascending=True).reset_index(drop=True)

df

,date,title,content
0,2008-04-10,금융통화위원회 의사록(2008년도 제8차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,2008-05-08,금융통화위원회 의사록(2008년도 제10차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,2008-05-22,금융통화위원회 의사록(2008년도 제11차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,2008-06-12,금융통화위원회 의사록(2008년도 제12차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,2008-06-26,금융통화위원회 의사록(2008년도 제13차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
...,...,...,...
250,2020-06-11,제13차_금통위 의사록,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
251,2020-06-24,제14차_금통위 의사록,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
252,2020-07-16,제15차_금통위 의사록,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
253,2020-07-17,제16차_금통위 의사록,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [36]:
df.to_csv('C:/Users/user/MPB_minutes/MPB_minutes.csv', encoding='utf-8-sig', header=True, index=False)

In [38]:
df.title[:20]

0      금융통화위원회 의사록(2008년도 제8차)
1     금융통화위원회 의사록(2008년도 제10차)
2     금융통화위원회 의사록(2008년도 제11차)
3     금융통화위원회 의사록(2008년도 제12차)
4     금융통화위원회 의사록(2008년도 제13차)
5     금융통화위원회 의사록(2008년도 제14차)
6     금융통화위원회 의사록(2008년도 제15차)
7     금융통화위원회 의사록(2008년도 제16차)
8     금융통화위원회 의사록(2008년도 제18차)
9     금융통화위원회 의사록(2008년도 제19차)
10                    munete20
11            제21차 금통위 의사록 (8)
12            제22차 금통위 의사록 (6)
13            제23차 금통위 의사록 (7)
14            제25차 금통위 의사록 (7)
15            제26차 금통위 의사록 (3)
16            제27차 금통위 의사록 (2)
17             제1차 금통위 의사록 (9)
18             제2차 금통위 의사록 (6)
19             제3차 금통위 의사록 (8)
Name: title, dtype: object